In [1]:
# Importaciones estándar
import os
import socket
import random
from itertools import islice

# Manejo de datos
import pandas as pd
import numpy as np

# Utilidades para limpieza de texto
import unicodedata
from unidecode import unidecode

# Visualización
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns

# Procesamiento y estadísticas
from tqdm import tqdm
from scipy.stats import boxcox
from sklearn.preprocessing import MinMaxScaler

#constantes
import constants
from constants import columna_valor_programa
from constants import columna_cupos_maximos
from constants import MINIMO_CUPOS_CERRADOS_SEDE

### consideraciones a incluir:

Entonces se debería volver a correr teniendo en cuenta:
1. Valores totales indexados
2. Mínimo de 50 para grupos cerrados y que no superen el máximo aprobado
3. El orden de priorizacion

In [2]:
#Valor total del programa indexado para realizar la asignación de cupos
columna_valor_programa

'VALOR TOTAL DEL PROGRAMA INDEXADO'

-----

In [3]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)

pd.set_option('display.float_format', '{:,.2f}'.format)

In [4]:
# Obtener el nombre del host
hostname = socket.gethostname()

hostname

'alejandro-lenovo'

In [5]:
# Obtener el nombre del host
hostname = socket.gethostname()

# Definir las rutas para cada computadora
path_pc1 = 'D:/OneDrive - Agencia Distrital para la Educación Superior, la Ciencia y la Tecnología - Atenea/Despacho/Investigaciones'
path_pc2 = 'C:/Users/JuliánNaranjo/OneDrive - Agencia Distrital para la Educación Superior, la Ciencia y la Tecnología - Atenea/2025 SAIGC/Índice ETDH'
path_pc3 = 'C:/Users/JULIA/OneDrive - Agencia Distrital para la Educación Superior, la Ciencia y la Tecnología - Atenea/Despacho/Investigaciones'

# Cambiar el directorio según el nombre del host
if hostname == 'DESKTOP-AEUL37C':
    os.chdir(path_pc1)
elif hostname == 'SGAINF-CND8085QVH-P26':
    os.chdir(path_pc2)
elif hostname == 'MSI':
    os.chdir(path_pc3)
else:
    print("Nombre de host no reconocido. Configura las rutas apropiadas para este dispositivo.")

# Imprime el directorio actual para verificar
print("Directorio actual:", os.getcwd())

Nombre de host no reconocido. Configura las rutas apropiadas para este dispositivo.
Directorio actual: /home/alejandro/Documentos/ATENEA/Despacho/Asignacion EFT


### Funciones

In [6]:
# Aplicar reemplazo seguro de los cno e ipo de los programas nuevos
def reemplazar_codigo(row):
    programa = row['nombre_programa']
    if programa in constants.programa_info:
        return pd.Series(constants.programa_info[programa])
    return pd.Series([row['cod_CNO'], row['IPO']])
    

In [7]:
def calcular_recursos_por_cno(df, total_recursos, group = ['cod_CNO','Ocupacion']):
    """
    Calcula los recursos por cno segun la formula del paso 1 de la ruta de antiguos y nuevos. 
    
    Parámetros:
        df (pd.DataFrame): DataFrame con columnas 'cod_CNO', 'Ocupacion', 'ipo_ponderado', 'numero_cupos_ofertar', 'IPO'.
        total_recursos (float): Total de recursos disponibles para distribuir.

    Retorna:
        pd.DataFrame: DataFrame con las métricas agrupadas y columna 'recursosxcno' calculada.
    """
    # 1. Calcular la suma total de ipo_ponderado
    IO_total = df['ipo_ponderado'].sum()

    # 2. Agrupar por cod_CNO y Ocupacion
    IO_por_CNO = (
        df
        .groupby(group)
        .agg(
            ipo_ponderado=('ipo_ponderado', 'sum'),
            CUPOS=('numero_cupos_ofertar', 'sum'),
            IPO=('IPO', 'sum'),
            n_programas=('ipo_ponderado', 'count')
        )
        .reset_index()
    )

    # 3. Calcular participación
    IO_por_CNO['IO_part'] = IO_por_CNO['ipo_ponderado'] / IO_total

    # 4. Asignar recursos
    IO_por_CNO['recursosxcno'] = IO_por_CNO['IO_part'] * total_recursos

    IO_por_CNO = IO_por_CNO[ group + ['recursosxcno','n_programas']]

    return IO_por_CNO


In [8]:
def ordenar_ocupaciones_por_isoeft(df):
    """
    Ordena un DataFrame por ['cod_CNO', 'Ocupacion', 'IPO', 'ISOEFT_4d'],
    asegurando que las filas con NaN en 'ISOEFT_4d' queden al final del DataFrame completo.

    Parámetros:
        df (pd.DataFrame): El DataFrame a ordenar.

    Retorna:
        pd.DataFrame: DataFrame ordenado con NaNs al final.
    """
    # Separar por presencia de NaN en ISOEFT_4d
    sin_nan = df[df['ISOEFT_4d'].notna()]
    con_nan = df[df['ISOEFT_4d'].isna()]

    #Columnas para ordenar
    columnas = [
        'IPO',
        'cod_CNO',
        'Ocupacion',
        'ISOEFT_4d',
        columna_valor_programa,
        "numero_cupos_ofertar",
        "duracion_horas_programa"
    ]

    orden = [
        False,
        True,
        True,
        False,
        True,
        False,
        True
    ]
    
    # Ordenar las filas válidas
    sin_nan = sin_nan.sort_values(
        columnas, 
        ascending= orden
    )

    # Concatenar y devolver
    return pd.concat([sin_nan, con_nan], ignore_index=True)


In [9]:
def ordenar_sedes_programas(df):
    """
    Ordena los programas dentro de cada ocupación (cod_CNO) siguiendo criterios:
    1. Mayor puntaje
    2. Mayor número de cupos ofertados
    3. Mayor meta de vinculación
    4. Menor costo
    5. Menor duración

    Parámetros:
        df (pd.DataFrame): DataFrame con columnas relevantes.

    Retorna:
        pd.DataFrame: DataFrame ordenado según los criterios establecidos.
    """
    columnas = [
        'cod_CNO',  # agrupación por ocupación
        'Puntaje (nuevos y cerrados)',
        'numero_cupos_ofertar',
        'Meta de vinculación',
        columna_valor_programa,
        'duracion_horas_programa'
    ]

    orden = [
        True,     # Para que agrupaciones se mantengan, aunque puedes omitir este en orden
        False,    # Mayor puntaje primero
        False,    # Más cupos primero
        False,    # Más meta de vinculación primero
        True,     # Menor costo primero
        True      # Menor duración primero
    ]

    df_ordenado = df.sort_values(by=columnas, ascending=orden).reset_index(drop=True)
    return df_ordenado


In [10]:
def asignar_recursos_y_cupos_viejos(data):
    """
    Asigna cupos y recursos por ocupación según los lineamientos de la Ruta Antigua, paso 2

    Retorna un resumen por ocupación con cupos y recursos asignados, y los saldos no utilizados.
    """
   
    data = data.copy()
    # Paso 1: Crear nueva columna para asignación
    data['cupos_asignados_2E'] = 0

    # Paso 2: Iterar por grupo de ocupación para asignar los recursos disponibles
    for (cod_cno, Ocupacion), grupo in data.groupby(['cod_CNO', 'Ocupacion']):
        recurso_por_dispersar = grupo['recursosxcno'].iloc[0]
        saldo = recurso_por_dispersar
        indices = grupo.index

        for i in indices:
            costo_unitario = data.loc[i, columna_valor_programa]
            cupos_disp = data.loc[i, 'numero_cupos_ofertar']

            if pd.isna(costo_unitario) or costo_unitario == 0:
                continue

            recurso_necesario = cupos_disp * costo_unitario

            if saldo >= recurso_necesario:
                data.loc[i, 'cupos_asignados_2E'] = cupos_disp
                saldo -= recurso_necesario
            else:
                # Ver si se puede financiar al menos un cupo
                cupos_asignables = saldo // costo_unitario
                data.loc[i, 'cupos_asignados_2E'] = cupos_asignables
                saldo -= cupos_asignables * costo_unitario
                break

    # Paso 3: Calcular recursos efectivamente asignados por programa
    data['recurso_asignado_2E'] = data['cupos_asignados_2E'] * data[columna_valor_programa]

    # Paso 4: Agrupar para obtener resumen de asignaciones por ocupacion
    asignacion_por_ocupacion_ant = data.groupby(['cod_CNO', 'Ocupacion']).agg(
        recurso_asignado_2E=('recurso_asignado_2E', 'sum'),
        cupos_asignados_2E=('cupos_asignados_2E', 'sum')
    ).reset_index()

    # Paso 5: Obtener recursos originales y número de cupos ofertados por las instituciones
    recursos_por_ocupacion = data.groupby(['cod_CNO', 'Ocupacion']).agg(
        recursosxcno=('recursosxcno', 'first'),
        numero_cupos_ofertar=('numero_cupos_ofertar', 'sum')
    ).reset_index()

    # Paso 6: Unir ambas tablas
    asignacion_por_ocupacion_ant = asignacion_por_ocupacion_ant.merge(
        recursos_por_ocupacion, on=['cod_CNO', 'Ocupacion']
    )

    # Paso 7: Calcular saldos no asignados
    asignacion_por_ocupacion_ant['Saldo_No_Asignado_2E'] = (
        asignacion_por_ocupacion_ant['recursosxcno'] - asignacion_por_ocupacion_ant['recurso_asignado_2E']
    )

    asignacion_por_ocupacion_ant['cupos_no_asignados_2E'] = (
        asignacion_por_ocupacion_ant['numero_cupos_ofertar'] - asignacion_por_ocupacion_ant['cupos_asignados_2E']
    )

    return data, asignacion_por_ocupacion_ant


In [11]:
def asignar_tercera_etapa(Antiguos, asignacion_por_ocupacion_ant):
    """
    Asigna recursos sobrantes de la segunda etapa a programas priorizados en una tercera etapa,
    usando una bolsa común. Actualiza el DataFrame original con asignaciones adicionales.
    """
    saldo_comun_3E = asignacion_por_ocupacion_ant['Saldo_No_Asignado_2E'].sum()
    
    Antiguos = Antiguos.copy()
    
    Antiguos['cupos_asignados_3E'] = 0
    Antiguos['recurso_asignado_3E'] = 0

    Antiguos = ordenar_ocupaciones_por_isoeft(Antiguos)

    for idx, row in Antiguos.iterrows():
        costo = row[columna_valor_programa]
        cupos = row['numero_cupos_ofertar']
        if pd.isna(costo) or costo <= 0 or pd.isna(cupos) or cupos <= 0:
            continue

        recurso_necesario = costo * cupos
        
        if saldo_comun_3E >= recurso_necesario:
            Antiguos.at[idx, 'cupos_asignados_3E'] = cupos
            Antiguos.at[idx, 'recurso_asignado_3E'] = recurso_necesario
            saldo_comun_3E -= recurso_necesario
        else:
            cupos_posibles = saldo_comun_3E // costo
            if cupos_posibles >= 1:
                recurso_asignado = cupos_posibles * costo
                Antiguos.at[idx, 'cupos_asignados_3E'] = cupos_posibles
                Antiguos.at[idx, 'recurso_asignado_3E'] = recurso_asignado
                saldo_comun_3E -= recurso_asignado
            else:
                break

    Antiguos['Total_Cupos_Asignados'] = Antiguos['cupos_asignados_2E'] + Antiguos['cupos_asignados_3E']
    Antiguos['Total_Recurso_Asignado'] = Antiguos['recurso_asignado_2E'] + Antiguos['recurso_asignado_3E']
    Antiguos['Saldo_Remanente_3E'] = saldo_comun_3E
    return Antiguos

In [12]:
def asignar_recursos_y_cupos_nuevos(data):
    
    data = data.copy()
    
    # Inicializar columna para cupos asignados
    data['cupos_asignados_2E'] = 0
    
    # Asignación iterativa por cod_CNO
    for cod_cno, grupo in data.groupby('cod_CNO'):
        
        recurso_total = grupo['recursosxcno'].iloc[0] #mismo recurso para cada ocupacion
        saldo = recurso_total
        indices = grupo.index
    
        for i in indices:
            costo_unitario = data.loc[i, columna_valor_programa]
            cupos_disp = data.loc[i, 'numero_cupos_ofertar']
    
            if pd.isna(costo_unitario) or costo_unitario == 0:
                continue
    
            recurso_necesario = cupos_disp * costo_unitario
    
            if saldo >= recurso_necesario:
                data.loc[i, 'cupos_asignados_2E'] = cupos_disp
                saldo -= recurso_necesario
            else:
                cupos_asignables = saldo // costo_unitario
                data.loc[i, 'cupos_asignados_2E'] = cupos_asignables
                saldo -= cupos_asignables * costo_unitario
                break
    
    # Paso 5: Calcular recursos asignados
    data['recurso_asignado_2E'] = data['cupos_asignados_2E'] * data[columna_valor_programa]
    
    # Paso 6: Agrupar para obtener resumen de asignaciones por ocupacion
    asignacion_por_ocupacion_nuevos = data.groupby('cod_CNO').agg(
        recurso_asignado_2E=('recurso_asignado_2E', 'sum'),
        cupos_asignados_2E=('cupos_asignados_2E', 'sum')
    ).reset_index()
    
    
    # Paso 5: Obtener recursos originales y número de cupos ofertados por las instituciones
    recursos_por_ocupacion = data.groupby('cod_CNO').agg(
        recursosxcno=('recursosxcno', 'first'),
        numero_cupos_ofertar=('numero_cupos_ofertar', 'sum')
    ).reset_index()
    
    
    # Paso 7: Agregar recursos estimados y calcular saldo no asignado
    asignacion_por_ocupacion_nuevos = asignacion_por_ocupacion_nuevos.merge(
        recursos_por_ocupacion, on='cod_CNO'
    )
    
    asignacion_por_ocupacion_nuevos['Saldo_No_Asignado_2E'] = (
        asignacion_por_ocupacion_nuevos['recursosxcno'] - asignacion_por_ocupacion_nuevos['recurso_asignado_2E']
    )
    
    asignacion_por_ocupacion_nuevos['cupos_no_asignados_2E'] = (
        asignacion_por_ocupacion_nuevos['numero_cupos_ofertar'] - asignacion_por_ocupacion_nuevos['cupos_asignados_2E']
    )

    return data, asignacion_por_ocupacion_nuevos

In [13]:
def asignar_recursos_grupos_cerrados(Grupos_Cerrados, total_recursos_cerrados):
    """
    Asigna recursos a programas de grupos cerrados hasta agotar el saldo total disponible.

    Parámetros:
        Grupos_Cerrados (pd.DataFrame): DataFrame con la información de los programas.
        total_recursos_cerrados (float): Monto total disponible para asignar.
        columna_valor_programa (str): Nombre de la columna con el valor unitario por cupo.

    Retorna:
        pd.DataFrame: DataFrame con columnas de cupos y recursos asignados, y el saldo restante.
    """
    Grupos_Cerrados = Grupos_Cerrados.copy()
    
    Grupos_Cerrados['cupos_asignados_2E'] = 0  
    Grupos_Cerrados['recurso_asignado_2E'] = 0.0
    Grupos_Cerrados['saldo_total_remanente'] = 0.0


    saldo_total = total_recursos_cerrados

    for idx, row in Grupos_Cerrados.iterrows():
        costo = row[columna_valor_programa]
        cupos = min(row['numero_cupos_ofertar'], row[columna_cupos_maximos])

        if pd.isna(costo) or costo <= 0 or pd.isna(cupos) or cupos <= 0:
            Grupos_Cerrados.at[idx, 'saldo_total_remanente'] = saldo_total
            continue

        recurso_necesario = costo * cupos

        if saldo_total >= recurso_necesario:
            Grupos_Cerrados.at[idx, 'cupos_asignados_2E'] = cupos
            Grupos_Cerrados.at[idx, 'recurso_asignado_2E'] = recurso_necesario
            saldo_total -= recurso_necesario
        else:
            
            cupos_posibles = saldo_total // costo
            recurso_asignado = cupos_posibles * costo
            
            Grupos_Cerrados.at[idx, 'cupos_asignados_2E'] = cupos_posibles
            Grupos_Cerrados.at[idx, 'recurso_asignado_2E'] = recurso_asignado
            saldo_total -= recurso_asignado

        Grupos_Cerrados.at[idx, 'saldo_total_remanente'] = saldo_total

        if saldo_total <= 0:
            break

    return Grupos_Cerrados, saldo_total

In [14]:
def asignar_recursos_hasta_validacion(Grupos_Cerrados, recursos_por_dispersar, MINIMO_CUPOS_CERRADOS_SEDE=50):
    """
    Ejecuta la asignación de recursos, eliminando iterativamente instituciones que no alcanzan el mínimo de cupos.

    Retorna:
        Grupos_Cerrados_final (pd.DataFrame)
        instituciones_eliminadas (list[str])
    """
    instituciones_eliminadas = [] #Aqui se guardaran las instituciones que no cumplieron con el minimo de 50 cupos
    filas_eliminadas = []  # Aquí se guardarán las filas eliminadas para concatenar al final
    continuar = True

    saldo_remanente = recursos_por_dispersar

    while continuar:
        # Asignar recursos
        Grupos_Cerrados, saldo_remanente = asignar_recursos_grupos_cerrados(Grupos_Cerrados, saldo_remanente)

        # Identificar instituciones a eliminar
        instituciones_a_eliminar = []

        for institucion in Grupos_Cerrados['nombre_institucion'].unique():
            df_inst = Grupos_Cerrados[Grupos_Cerrados['nombre_institucion'] == institucion]
            
            #evaluar la suma de los cupos asignados a la institucion
            total_cupos = df_inst['cupos_asignados_2E'].sum()
            #evaluar la suma dispersada a la institucion
            recursos_disperados_inst = df_inst['recurso_asignado_2E'].sum()
            
            if 0 < total_cupos < MINIMO_CUPOS_CERRADOS_SEDE:
                
                instituciones_a_eliminar.append(institucion)
                #reintregamos al saldolos recursos que se le habia dado a la institucion
                saldo_remanente += recursos_disperados_inst

        if instituciones_a_eliminar:
            
            # Seleccionar filas a eliminar
            filas_a_eliminar = Grupos_Cerrados[Grupos_Cerrados['nombre_institucion'].isin(instituciones_a_eliminar)].copy()
            
            # Actualizar sus valores
            filas_a_eliminar['cupos_asignados_2E'] = 0
            filas_a_eliminar['recurso_asignado_2E'] = 0.0
            filas_a_eliminar['saldo_total_remanente'] = saldo_remanente
            
            # Guardar filas actualizadas
            filas_eliminadas.append(filas_a_eliminar)

            # Añadir las instituciones a eliminar a la lista
            instituciones_eliminadas.extend(instituciones_a_eliminar)
            
            # Eliminar las instituciones del DataFrame principal
            Grupos_Cerrados = Grupos_Cerrados[
                ~Grupos_Cerrados['nombre_institucion'].isin(instituciones_a_eliminar)
            ].reset_index(drop=True)
        else:
            continuar = False

    # Reagregar las filas eliminadas al final del DataFrame
    if filas_eliminadas:
        Grupos_Cerrados = pd.concat([Grupos_Cerrados] + filas_eliminadas, ignore_index=True)

    return Grupos_Cerrados, instituciones_eliminadas


In [15]:
def reasignar_remanente(df_remanente, bolsa, columna_cupos= 'cupos_asignados_2E'):
    """
    Reasigna recursos disponibles (remanente) a los cupos restantes de un DataFrame, 
    hasta agotar el saldo o completar los cupos pendientes.
    """
    df_remanente = df_remanente.copy()
    
    #df_remanente['remanente_asignado'] = 0
    
    saldo_total = bolsa

    for idx, row in df_remanente.iterrows():
        cupos_restantes =  row['numero_cupos_ofertar'] - row[columna_cupos] 
        costo = row[columna_valor_programa]

        if pd.isna(costo) or costo <= 0 or pd.isna(cupos_restantes) or cupos_restantes <= 0:
            df_remanente.at[idx, 'saldo_total_remanente'] = saldo_total
            continue

        recurso_necesario = costo * cupos_restantes

        if saldo_total >= recurso_necesario:
            df_remanente.at[idx, 'cupos_asignados_remanente'] = cupos_restantes
            df_remanente.at[idx, 'recurso_asignado_remanente'] = recurso_necesario
            saldo_total -= recurso_necesario
        else:
            cupos_posibles = saldo_total // costo
            recurso_asignado = cupos_posibles * costo

            df_remanente.at[idx, 'cupos_asignados_remanente'] = cupos_posibles
            df_remanente.at[idx, 'recurso_asignado_remanente'] = recurso_asignado
            saldo_total -= recurso_asignado

        df_remanente.at[idx, 'saldo_total_remanente'] = saldo_total

        if saldo_total <= 0:
            break

    return df_remanente, saldo_total

### Lectura de datos

In [16]:
#Cargar los datos
Programas_EFT_Oferta = pd.read_pickle("Asignación de cupos/Base final - Oferta Activa.pkl")
Programas_EFT = pd.read_excel("Asignación de cupos/Habilitados final 26052025.xlsx")

In [17]:
#Limpieza de los datos

# Seleccionar las columnas de interes
Programas_EFT_Oferta = Programas_EFT_Oferta[constants.columnas_programas_eft_oferta]

#Renombrar algunas columnas
Programas_EFT = Programas_EFT.rename(
    columns= constants.nombre_columnas_mapping
)

#Cambiar el tipo de los datos
Programas_EFT = Programas_EFT.astype(constants.tipo_columnas_mapping)
Programas_EFT_Oferta = Programas_EFT_Oferta.astype(constants.tipo_columnas_mapping)

In [18]:
# Unir Programas_EFT y Programas_EFT_Oferta
Programas_EFT = Programas_EFT.merge(
    Programas_EFT_Oferta,
    how='left',
    on=['CODIGO_PROGRAMA']
)

----

### Descripcion de los datos

In [19]:
#CNO: Clasificacion nancional de ocupaciones
# Suponiendo que 'cod_CNO' proviene de Programas_EFT_Oferta y no está en Programas_EFT original
cruzaron = Programas_EFT['cod_CNO'].notna().sum()
no_cruzaron = Programas_EFT['cod_CNO'].isna().sum()

print(f"Registros que cruzaron: {cruzaron}")
print(f"Registros que NO cruzaron: {no_cruzaron}")

Registros que cruzaron: 39
Registros que NO cruzaron: 6


In [20]:
sin_cno_antiguos = Programas_EFT[
    Programas_EFT['cod_CNO'].isna() &
    (Programas_EFT['Ruta habilitada'] == "Antiguos")
]

print(f"Numero de programas antiguos sin cno: {sin_cno_antiguos.shape[0]}")

Numero de programas antiguos sin cno: 0


In [21]:
Programas_EFT['Ruta habilitada'].value_counts()

Ruta habilitada
Antiguos           25
Nuevos             10
Grupos cerrados    10
Name: count, dtype: int64

In [22]:
ruta_labels = Programas_EFT['Ruta habilitada'].unique()

# Crear un diccionario con un DataFrame por cada valor de ruta
rutas_dict = {
    ruta: Programas_EFT.loc[Programas_EFT['Ruta habilitada'] == ruta].copy()
    for ruta in ruta_labels
}

# Ejemplos de acceso:
Antiguos = rutas_dict.get("Antiguos")
Nuevos = rutas_dict.get("Nuevos")
No_Habilitado = rutas_dict.get("NO HABILITADO")
Grupos_Cerrados = rutas_dict.get("Grupos cerrados")

-----

# Ruta: Antiguos

Pasos:
1. PASO 1: Asignación por ocupación proporcional al IPO Ponderado
2. PASO 2: Asignación de recursos en función del resultado de ISOEFT
3. PASO 3: Bolsa común con recursos excedentes para completar recursos.

In [23]:
#Validación de que no existan ocupaciones con IPO menor al minimo (ie., 0.47)
min_value = Antiguos['IPO'].min()
print("Min:", min_value)
assert min_value >= 0.47, f"Min value {min_value} is less than 0.47"

Min: 0.4773236184370968


### Paso 1: Asignación por ocupación proporcional al IPO Ponderado

In [24]:
assert Antiguos['IPO'].notna().all(), "Existen NaNs in IPO"

In [25]:
assert Antiguos['numero_cupos_ofertar'].notna().all(), "Existen NaNs in 'numero_cupos_ofertar'"

In [26]:
# Calculo del IPO ponderado
Antiguos['ipo_ponderado'] = Antiguos['numero_cupos_ofertar'] * Antiguos['IPO'] 

In [27]:
total_recursos_antiguos = constants.recursos_por_ruta['antiguos']
print(f"Recursos totales para disperar en la ruta de Antiguos: {total_recursos_antiguos:,.0f}")

Recursos totales para disperar en la ruta de Antiguos: 1,320,000,000


In [28]:
IO_por_CNO = calcular_recursos_por_cno(Antiguos, total_recursos_antiguos)
IO_por_CNO.head()

,cod_CNO,Ocupacion,recursosxcno,n_programas
0,1231,Asistentes contables,"107,569,094.89",2
1,1345,Auxiliares administrativos en salud,"111,126,632.02",2
2,2242,Técnicos en electrónica,"45,308,756.34",1
3,2281,Técnicos en tecnologías de la información,"61,535,606.34",1
4,3311,Auxiliares en enfermería,"53,180,231.95",1


In [29]:
suma_recursos_por_cno = IO_por_CNO['recursosxcno'].sum()
print(f"Total de Recursos por CNO: {suma_recursos_por_cno:,.0f}")

# verificar que los recursos asignados por ocupaciones sean iguales a los recursos disponbiles
assert suma_recursos_por_cno == total_recursos_antiguos , f"Los recursos dispersados no son iguales al total de recursos disponibles"

Total de Recursos por CNO: 1,320,000,000


In [30]:
#Agregarle la columna recrusosxcno al df Antiguos
# Nota: Puede haber mas de un programa en una ocupacion, recursosxcno tiene valores duplicados
Antiguos = Antiguos.merge(
    # Le agreg
    IO_por_CNO[['cod_CNO', 'Ocupacion','recursosxcno']],
    on = ['cod_CNO','Ocupacion'],
    how='left'
)

### PASO 2: Asignación de recursos en función del resultado de ISOEFT

a) Ordenamiento de programas: Los programas dentro de cada ocupación se organizan en orden descendente, según su puntaje de desempeño en el ISOEFT, priorizando aquellos con mejor desempeño.

b) Asignación secuencial de recursos: La distribución de los recursos se realiza de manera ordenada, asignándolos a los programas ordenados y agrupados Este proceso sigue una secuencia estructurada en la que los cupos se asignan hasta que ocurra alguna de las siguientes condiciones:

- Se agoten los cupos ofertados dentro de la ocupación
- Se agoten los recursos asignados para la ocupación
- El saldo disponible no sea suficiente para financiar un cupo completo en ningún programa de la ocupación (siguiendo el ordenamiento por ISOEFT del programa)

c) Si, al finalizar el proceso, quedan recursos sin asignar, estos se trasladarán a una bolsa común, que será distribuida en la siguiente fase del proceso de asignación de recursos (Paso 3)

**a)** Ejecutar paso 2.a

El manual no es claro cual ocupación va primero:

"Ordenamiento de programas: Los programas dentro de cada ocupación se organizan en orden descendente, según su puntaje de desempeño en el ISOEFT, priorizando aquellos con mejor desempeño".

In [31]:
assert Antiguos['ISOEFT_4d'].notna().all(), "Existen NaNs in ISOEFT_4d"

AssertionError: Existen NaNs in ISOEFT_4d

In [32]:
assert Antiguos[columna_valor_programa].notna().all(), "Existen NaNs in valor programa indexado"

In [33]:
Antiguos = ordenar_ocupaciones_por_isoeft(Antiguos)

In [34]:
columnas_mostrar = [
    'cod_CNO',
    'Ocupacion',
    'IPO',
    'ISOEFT_4d',
    'CODIGO_INSTITUCION',
    'CODIGO_PROGRAMA',
    'nombre_programa',
    'recursosxcno', 
    'numero_cupos_ofertar'
]
Antiguos[columnas_mostrar].head()

,cod_CNO,Ocupacion,IPO,ISOEFT_4d,CODIGO_INSTITUCION,CODIGO_PROGRAMA,nombre_programa,recursosxcno,numero_cupos_ofertar
0,2281,Técnicos en tecnologías de la información,0.67,0.39,9025,42561,TÉCNICO LABORAL EN PROCESAMIENTO Y DIGITACIÓN ...,"61,535,606.34",83
1,1345,Auxiliares administrativos en salud,0.62,0.61,975,43512,TÉCNICO LABORAL EN AUXILIAR ADMINISTRATIVO EN ...,"111,126,632.02",80
2,1345,Auxiliares administrativos en salud,0.62,0.53,1236,41020,TÉCNICO LABORAL EN AUXILIAR ADMINISTRATIVO EN ...,"111,126,632.02",80
3,3315,Auxiliares en servicios farmacéuticos,0.60,0.54,1236,41022,TÉCNICO LABORAL EN AUXILIAR EN SERVICIOS FARMA...,"106,929,561.91",80
4,3315,Auxiliares en servicios farmacéuticos,0.60,0.53,975,43511,TÉCNICO LABORAL EN AUXILIAR EN SERVICIOS FARMA...,"106,929,561.91",80


**b)** Ejecutar Paso 2.b

In [35]:
Antiguos, asignacion_por_ocupacion_ant = asignar_recursos_y_cupos_viejos(Antiguos)

In [36]:
asignacion_por_ocupacion_ant.head()

,cod_CNO,Ocupacion,recurso_asignado_2E,cupos_asignados_2E,recursosxcno,numero_cupos_ofertar,Saldo_No_Asignado_2E,cupos_no_asignados_2E
0,1231,Asistentes contables,"106,624,039.03",29,"107,569,094.89",200,"945,055.86",171
1,1345,Auxiliares administrativos en salud,"110,572,617.97",17,"111,126,632.02",160,"554,014.05",143
2,2242,Técnicos en electrónica,"42,623,849.96",10,"45,308,756.34",75,"2,684,906.38",65
3,2281,Técnicos en tecnologías de la información,"57,725,449.20",12,"61,535,606.34",83,"3,810,157.14",71
4,3311,Auxiliares en enfermería,"49,340,892.24",5,"53,180,231.95",100,"3,839,339.71",95


### PASO 3: Bolsa común con recursos excedentes para completar recursos

Los recursos excedentes serán agrupados en una bolsa común y asignados siguiendo lo estipulado en el paso 2. Primero, se ordenarán las ocupaciones según el IPO, y dentro de cada ocupación, los programas se priorizarán por su ISOEFT. La asignación de recursos continuará siguiendo este orden hasta que se financien todos los cupos de un programa, se agoten los recursos de la bolsa común o el saldo disponible no permita financiar al menos un cupo completo.

Criterios de priorización:
En caso de que haya programas que tengan el mismo valor del ISOEFT, los criterios de priorización para la asignación de recursos se realizarán en el siguiente orden:

- i. Se dará preferencia a los programas con el menor valor de matrícula
- ii. En el caso de que tengan el mismo valor de matrícula se priorizará el programa para el cual se hayan ofertado más cupos.
- iii. En caso de los programas tengan el mismo valor de matrícula y el mismo número de cupos ofertados, se priorizará el programa con el menor número de horas.

In [37]:
assert Antiguos[columna_valor_programa].notna().all(), "Existen NaNs in valor programa indexado"

In [38]:
Antiguos = asignar_tercera_etapa(Antiguos, asignacion_por_ocupacion_ant)

/tmp/ipykernel_31833/3855908659.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '28862724.599999998' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Antiguos.at[idx, 'recurso_asignado_3E'] = recurso_asignado


In [39]:
columnas_mostrar = [
    'cod_CNO',
    'Ocupacion',
    'IPO',
    'ISOEFT_4d',
    'CODIGO_INSTITUCION',
    'CODIGO_PROGRAMA',
    'recursosxcno', 
    'numero_cupos_ofertar',
    'cupos_asignados_2E',
    'recurso_asignado_2E',
    'cupos_asignados_3E',
    'recurso_asignado_3E',
    'Total_Cupos_Asignados',
    "Saldo_Remanente_3E"
]
Antiguos[columnas_mostrar].head()

,cod_CNO,Ocupacion,IPO,ISOEFT_4d,CODIGO_INSTITUCION,CODIGO_PROGRAMA,recursosxcno,numero_cupos_ofertar,cupos_asignados_2E,recurso_asignado_2E,cupos_asignados_3E,recurso_asignado_3E,Total_Cupos_Asignados,Saldo_Remanente_3E
0,2281,Técnicos en tecnologías de la información,0.67,0.39,9025,42561,"61,535,606.34",83,12,"57,725,449.20",6,"28,862,724.60",18,"4,390,248.13"
1,1345,Auxiliares administrativos en salud,0.62,0.61,975,43512,"111,126,632.02",80,17,"110,572,617.97",0,0.00,17,"4,390,248.13"
2,1345,Auxiliares administrativos en salud,0.62,0.53,1236,41020,"111,126,632.02",80,0,0.00,0,0.00,0,"4,390,248.13"
3,3315,Auxiliares en servicios farmacéuticos,0.60,0.54,1236,41022,"106,929,561.91",80,16,"106,447,062.42",0,0.00,16,"4,390,248.13"
4,3315,Auxiliares en servicios farmacéuticos,0.60,0.53,975,43511,"106,929,561.91",80,0,0.00,0,0.00,0,"4,390,248.13"


In [40]:
Antiguos.to_excel("Export/Antiguos.xlsx", index=False)
asignacion_por_ocupacion_ant.to_excel("Export/AntiguosxOcupación.xlsx", index=False)

------

# Ruta: Nuevos

Pasos:
1. PASO 1: Asignación por ocupación proporcional al IPO Ponderado
2. PASO 2: Asignación de puntaje a partir de los elementos puntuables
3. PASO 3: Asignación de recursos para la selección de cupos

In [41]:
#Limpieza datos

#reemplazar los codigos cno e ipo de manera segura
Nuevos[['cod_CNO', 'IPO']] = Nuevos.apply(reemplazar_codigo, axis=1)
# Forzar tipo entero en cod_CNO (permitiendo nulos)
Nuevos['cod_CNO'] = pd.to_numeric(Nuevos['cod_CNO'], errors='coerce').astype('Int64')

In [42]:
total_recursos_nuevos = constants.recursos_por_ruta['nuevos']
print(f"Recursos totales para disperar en la ruta de Nuevos: {total_recursos_nuevos:,.0f}")

Recursos totales para disperar en la ruta de Nuevos: 990,000,000


### Paso 1 

In [43]:
Nuevos['ipo_ponderado'] = Nuevos['numero_cupos_ofertar'] * Nuevos['IPO'] 

In [44]:
IO_por_CNO_nuevo = calcular_recursos_por_cno(Nuevos, total_recursos_nuevos, group = ['cod_CNO'])
IO_por_CNO_nuevo

,cod_CNO,recursosxcno,n_programas
0,2281,"589,967,275.83",4
1,2321,"69,818,825.16",1
2,3311,"60,442,068.24",1
3,6233,"66,270,863.08",1
4,6374,"70,705,815.68",1
5,6642,"69,185,260.50",1
6,8325,"63,609,891.52",1


In [45]:
suma_recursos_por_cno = IO_por_CNO_nuevo['recursosxcno'].sum()
print(f"Total de Recursos para dispersar: {suma_recursos_por_cno:,.0f}")

# verificar que los recursos asignados por ocupaciones sean iguales a los recursos disponbiles
assert suma_recursos_por_cno == total_recursos_nuevos , f"Los recursos dispersados no son iguales al total de recursos disponibles"

Total de Recursos para dispersar: 990,000,000


In [46]:
Nuevos = Nuevos.merge(
    IO_por_CNO_nuevo[['cod_CNO','recursosxcno']],
    on=['cod_CNO'],
    how='left'
)

### Paso 2
La data viene con la columna de puntajes

### Paso 3

In [47]:
# Paso 2: Ordenar programas por prioridad (dentro de cada cod_CNO)
Nuevos =  ordenar_sedes_programas(Nuevos)

In [48]:
Nuevos, asignacion_por_ocupacion_nuevos = asignar_recursos_y_cupos_nuevos(Nuevos)

In [49]:
Nuevos.to_excel("Export/Nuevos.xlsx", index=False)
asignacion_por_ocupacion_nuevos.to_excel("Export/NuevosxOcupación.xlsx", index=False)

-----

### Grupos Cerrados

### Paso 1 

La data viene con los puntajes

### Paso 2: Asignación de recursos para la selección de cupos

In [50]:
# Ordenar programas por prioridad
Grupos_Cerrados = ordenar_sedes_programas(Grupos_Cerrados)

In [51]:
total_recursos_cerrados = constants.recursos_por_ruta['cerrados']
print(f"Recursos totales para disperar en la ruta de Cerrados: {total_recursos_cerrados:,.0f}")

Recursos totales para disperar en la ruta de Cerrados: 990,000,000


In [52]:
Grupos_Cerrados_final, instituciones_fuera = asignar_recursos_hasta_validacion(
    Grupos_Cerrados,
    total_recursos_cerrados,
    MINIMO_CUPOS_CERRADOS_SEDE=50  # o el valor que definas
)

In [53]:
Grupos_Cerrados_final[['nombre_institucion',columna_valor_programa,columna_cupos_maximos,'cupos_asignados_2E', 'recurso_asignado_2E', 'saldo_total_remanente']]

,nombre_institucion,VALOR TOTAL DEL PROGRAMA INDEXADO,Número máximo de cupos por grupos,cupos_asignados_2E,recurso_asignado_2E,saldo_total_remanente
0,KUEPA EDUTECH,"4,810,454.10",888,110,"529,149,951.00","460,850,049.00"
1,CESDE BOGOTÁ,"6,232,522.68",90,73,"454,974,155.57","5,875,893.43"
2,MEDISED INSTITUCION DE EDUCACION PARA EL TRABA...,"6,839,631.92",100,0,0.00,"5,875,893.43"
3,FEE ESTUDIO EMPRESARIAL - CHAPINERO,"6,504,271.65",420,0,0.00,"5,875,893.43"
4,FEE ESTUDIO EMPRESARIAL - TEUSAQUILLO,"6,652,941.40",612,0,0.00,"5,875,893.43"
5,FEE ESTUDIO EMPRESARIAL - CHAPINERO,"6,194,543.61",420,0,0.00,"5,875,893.43"
6,FEE ESTUDIO EMPRESARIAL - TEUSAQUILLO,"6,652,941.40",612,0,0.00,"5,875,893.43"
7,KUEPA EDUTECH,"4,810,454.10",277,1,"4,810,454.10","1,065,439.33"
8,KUEPA EDUTECH,"4,810,454.10",69,0,0.00,"1,065,439.33"
9,CESDE BOGOTÁ,"5,406,154.01",60,0,0.00,"1,065,439.33"


In [54]:
recurso_asignado_cerrados = Grupos_Cerrados_final[['recurso_asignado_2E']].sum()
print(recurso_asignado_cerrados)

recurso_asignado_2E   988,934,560.67
dtype: float64


In [55]:
Grupos_Cerrados_final.to_excel("Export/GCerrados.xlsx", index=False)

----

# Reasignacion del Remanente

In [56]:
recursos_remanente_antiguos = Antiguos['Saldo_Remanente_3E'].loc[0] 
print(f"Bolsa remanente Antiguos: {recursos_remanente_antiguos:,.0f}")
recursos_remanente_nuevos = asignacion_por_ocupacion_nuevos['Saldo_No_Asignado_2E'].sum()
print(f"Bolsa remanente Nuevos: {recursos_remanente_nuevos:,.0f}")
recursos_remanente_cerrados = Grupos_Cerrados_final.iloc[-1]["saldo_total_remanente"]
print(f"Bolsa remanente Cerrados: {recursos_remanente_cerrados:,.0f}")
bolsa_comun = recursos_remanente_antiguos + recursos_remanente_nuevos + recursos_remanente_cerrados

Bolsa remanente Antiguos: 4,390,248
Bolsa remanente Nuevos: 17,398,911
Bolsa remanente Cerrados: 1,065,439


In [57]:
print(f"Bolsa remanente: {bolsa_comun:,.0f}")

Bolsa remanente: 22,854,598


In [58]:
#identificar los programas que aun no han llenado cupos:
Grupos_Cerrados_remanente = Grupos_Cerrados_final[
    Grupos_Cerrados_final['numero_cupos_ofertar'] - Grupos_Cerrados_final['cupos_asignados_2E'] > 0
].reset_index(drop=True)

Grupos_Cerrados_remanente = ordenar_sedes_programas(Grupos_Cerrados_remanente)

Nuevos_remanente = Nuevos[
    Nuevos['numero_cupos_ofertar'] - Nuevos['cupos_asignados_2E'] > 0
].reset_index(drop=True)

Nuevos_remanente = ordenar_sedes_programas(Nuevos_remanente)

Antiguos_remanente = Antiguos[
    Antiguos['numero_cupos_ofertar'] - Antiguos['Total_Cupos_Asignados'] > 0
].reset_index(drop=True)

Antiguos_remanente = ordenar_ocupaciones_por_isoeft(Antiguos_remanente)

In [59]:
Grupos_Cerrados_final[['nombre_institucion',columna_valor_programa,columna_cupos_maximos,'cupos_asignados_2E', 'recurso_asignado_2E', 'saldo_total_remanente']]

,nombre_institucion,VALOR TOTAL DEL PROGRAMA INDEXADO,Número máximo de cupos por grupos,cupos_asignados_2E,recurso_asignado_2E,saldo_total_remanente
0,KUEPA EDUTECH,"4,810,454.10",888,110,"529,149,951.00","460,850,049.00"
1,CESDE BOGOTÁ,"6,232,522.68",90,73,"454,974,155.57","5,875,893.43"
2,MEDISED INSTITUCION DE EDUCACION PARA EL TRABA...,"6,839,631.92",100,0,0.00,"5,875,893.43"
3,FEE ESTUDIO EMPRESARIAL - CHAPINERO,"6,504,271.65",420,0,0.00,"5,875,893.43"
4,FEE ESTUDIO EMPRESARIAL - TEUSAQUILLO,"6,652,941.40",612,0,0.00,"5,875,893.43"
5,FEE ESTUDIO EMPRESARIAL - CHAPINERO,"6,194,543.61",420,0,0.00,"5,875,893.43"
6,FEE ESTUDIO EMPRESARIAL - TEUSAQUILLO,"6,652,941.40",612,0,0.00,"5,875,893.43"
7,KUEPA EDUTECH,"4,810,454.10",277,1,"4,810,454.10","1,065,439.33"
8,KUEPA EDUTECH,"4,810,454.10",69,0,0.00,"1,065,439.33"
9,CESDE BOGOTÁ,"5,406,154.01",60,0,0.00,"1,065,439.33"


In [60]:
#llenar cupos de cerrados manteniendo la condicion que una institucion no puede tener menos de 50 cupos
Grupos_Cerrados_remanente, instituciones_fuera = asignar_recursos_hasta_validacion(
    Grupos_Cerrados_remanente,
    bolsa_comun,
    MINIMO_CUPOS_CERRADOS_SEDE=50  # o el valor que definas
)
#"saldo_total_remanente"

bolsa_comun = Grupos_Cerrados_remanente['saldo_total_remanente'].iloc[-1]

In [61]:
print(f"Bolsa remanente: {bolsa_comun:,.0f}")

Bolsa remanente: 22,854,598


In [62]:
Nuevos_remanente, bolsa_comun = reasignar_remanente(Nuevos_remanente,bolsa_comun)
print(f"Bolsa remanente: {bolsa_comun:,.0f}")

Bolsa remanente: 2,570,517


In [63]:
Antiguos_remanente, bolsa_comun = reasignar_remanente(Antiguos_remanente, bolsa_comun, columna_cupos = "cupos_asignados_3E")
print(f"Bolsa remanente: {bolsa_comun:,.0f}")

Bolsa remanente: 881,793


---------

In [87]:
def unir_con_remanente_v2(df_original, df_remanente, llaves=['cod_CNO', 'nombre_programa', 'nombre_institucion']):
    """
    Devuelve un DataFrame combinado que prioriza las filas actualizadas del remanente
    y conserva las filas no modificadas del original, eliminando duplicados por llaves.

    Parámetros:
        df_original (pd.DataFrame): DataFrame original antes de la reasignación.
        df_remanente (pd.DataFrame): DataFrame después de la reasignación.
        llaves (list[str]): Columnas clave para identificar filas únicas.

    Retorna:
        pd.DataFrame: DataFrame combinado sin duplicados en las llaves.
    """
    combinado = pd.concat([df_original, df_remanente ], ignore_index=True)
    df_final = combinado.drop_duplicates(subset=llaves, keep='first')
    return df_final


In [81]:
def unir_con_remanente(df_original, df_remanente, llaves = ['cod_CNO', 'nombre_programa', 'nombre_institucion']):
    """
    Devuelve un DataFrame final que contiene las filas actualizadas del remanente y
    las filas del original que no fueron modificadas.

    Parámetros:
        df_original (pd.DataFrame): DataFrame original antes de la reasignación.
        df_remanente (pd.DataFrame): DataFrame después de la reasignación.
        claves (list[str]): Lista de columnas clave para identificar registros únicos.

    Retorna:
        pd.DataFrame: DataFrame combinado con actualizados + no modificados.
    """
    
    # Merge para identificar filas de df_original que NO están en df_remanente
    df_merge = df_original.merge(
        df_remanente[llaves],
        on=llaves,
        indicator=True
    )
    
    # Filtrar filas únicas del original (no están en remanente)
    df_diff = df_merge.query('_merge == "left_only"').drop(columns=['_merge'])
    
    # Opcional: imprimir las filas faltantes para revisión
    #print("Filas en original no encontradas en remanente:")
    #print(df_diff[llaves])
    
    # Concatenar remanente con filas no modificadas del original
    df_final = pd.concat([df_remanente, df_diff], ignore_index=True)

    return df_final



In [93]:
llaves = ['cod_CNO', 'nombre_institucion','nombre_programa']

Nuevos_consolidado = unir_con_remanente(Nuevos, Nuevos_remanente, llaves)
Grupos_Cerrados_consolidado = unir_con_remanente_v2(Grupos_Cerrados_final, Grupos_Cerrados_remanente, llaves = llaves)
Antiguos_consolidado = unir_con_remanente(Antiguos, Antiguos_remanente, llaves)


In [89]:
Grupos_Cerrados_consolidado[['nombre_institucion',columna_valor_programa,columna_cupos_maximos,'cupos_asignados_2E', 'recurso_asignado_2E', 'saldo_total_remanente']]

,nombre_institucion,VALOR TOTAL DEL PROGRAMA INDEXADO,Número máximo de cupos por grupos,cupos_asignados_2E,recurso_asignado_2E,saldo_total_remanente
0,KUEPA EDUTECH,"4,810,454.10",888,110,"529,149,951.00","460,850,049.00"
1,CESDE BOGOTÁ,"6,232,522.68",90,73,"454,974,155.57","5,875,893.43"
2,MEDISED INSTITUCION DE EDUCACION PARA EL TRABA...,"6,839,631.92",100,0,0.00,"5,875,893.43"
3,FEE ESTUDIO EMPRESARIAL - CHAPINERO,"6,504,271.65",420,0,0.00,"5,875,893.43"
4,FEE ESTUDIO EMPRESARIAL - TEUSAQUILLO,"6,652,941.40",612,0,0.00,"5,875,893.43"
5,FEE ESTUDIO EMPRESARIAL - CHAPINERO,"6,194,543.61",420,0,0.00,"5,875,893.43"
6,FEE ESTUDIO EMPRESARIAL - TEUSAQUILLO,"6,652,941.40",612,0,0.00,"5,875,893.43"
7,KUEPA EDUTECH,"4,810,454.10",277,1,"4,810,454.10","1,065,439.33"
8,KUEPA EDUTECH,"4,810,454.10",69,0,0.00,"1,065,439.33"
9,CESDE BOGOTÁ,"5,406,154.01",60,0,0.00,"1,065,439.33"


In [110]:
#Antiguos
asignado_antiguos = Antiguos_consolidado['Total_Recurso_Asignado'].sum() + Antiguos_consolidado["recurso_asignado_remanente"].sum()
Antiguos_consolidado['total_dispersado'] = Antiguos_consolidado['Total_Recurso_Asignado'].fillna(0) + Antiguos_consolidado["recurso_asignado_remanente"].fillna(0)
Antiguos_consolidado['total_cupos_habilitados'] = Antiguos_consolidado['Total_Cupos_Asignados'].fillna(0) + Antiguos_consolidado["cupos_asignados_remanente"].fillna(0)
print(f"Asignado Antiguos: {asignado_antiguos:,.0f}")

#CERRADOS
asignado_cerrados = Grupos_Cerrados_consolidado['recurso_asignado_2E'].sum()
Grupos_Cerrados_consolidado['total_dispersado'] = Grupos_Cerrados_consolidado['recurso_asignado_2E'].fillna(0)
Grupos_Cerrados_consolidado['total_cupos_habilitados'] = Grupos_Cerrados_consolidado['cupos_asignados_2E'].fillna(0)
print(f"Asignado cerrados: {asignado_cerrados:,.0f}")

#NUEVOS
asignado_nuevos = Nuevos_consolidado["recurso_asignado_remanente"].sum()  + Nuevos_consolidado['recurso_asignado_2E'].sum()
Nuevos_consolidado['total_dispersado'] = Nuevos_consolidado["recurso_asignado_remanente"].fillna(0) + Nuevos_consolidado['recurso_asignado_2E'].fillna(0)
Nuevos_consolidado['total_cupos_habilitados'] = Nuevos_consolidado["cupos_asignados_remanente"].fillna(0) + Nuevos_consolidado['cupos_asignados_2E'].fillna(0)
print(f"Asignado nuevos: {asignado_nuevos:,.0f}")

total = asignado_antiguos +  asignado_cerrados  + asignado_nuevos
print(f"Asignado total: {total:,.0f}")

Asignado Antiguos: 1,317,298,476
Asignado cerrados: 988,934,561
Asignado nuevos: 992,885,171
Asignado total: 3,299,118,207


------

In [112]:
Grupos_Cerrados_consolidado.to_excel("Export/cerrados_consolidado.xlsx", index=False)
Antiguos_consolidado.to_excel("Export/antiguos_consolidado.xlsx", index=False)
Nuevos_consolidado.to_excel("Export/nuevos_consolidado.xlsx", index=False)

In [126]:
# Obtener las columnas comunes entre todos los DataFrames
#columnas_exportar = columnas_exportar = list(Programas_EFT.columns) + ['total_dispersado', 'total_cupos_habilitados']

columnas_exportar = [
    'nombre_institucion',
    'cod_CNO',
    'nombre_programa',
    "CODIGO_PROGRAMA",
    'Puntaje (nuevos y cerrados)',
    "numero_cupos_ofertar",
    columna_valor_programa,
    'duracion_horas_programa',
    'Ruta habilitada',
    'total_dispersado',
    'total_cupos_habilitados',
    "Número máximo de cupos por grupos",
]

# Filtrar solo esas columnas en cada DataFrame
df_grupos = Grupos_Cerrados_consolidado[columnas_exportar]
df_antiguos = Antiguos_consolidado[columnas_exportar]
df_nuevos = Nuevos_consolidado[columnas_exportar]

# Concatenar
df_consolidado = pd.concat([df_grupos, df_antiguos, df_nuevos], ignore_index=True)

# Exportar a Excel si lo deseas
df_consolidado.to_excel("Export/todos_consolidados.xlsx", index=False)

In [133]:
df_consolidado[columnas]

,nombre_institucion,cod_CNO,nombre_programa,CODIGO_PROGRAMA,Puntaje (nuevos y cerrados),numero_cupos_ofertar,VALOR TOTAL DEL PROGRAMA INDEXADO,duracion_horas_programa,Ruta habilitada,total_dispersado,total_cupos_habilitados,Número máximo de cupos por grupos
0,KUEPA EDUTECH,1341,TÉCNICO LABORAL EN AUXILIAR ADMINISTRATIVO,42558,88.75,110,"4,810,454.10",720,Grupos cerrados,"529,149,951.00",110.00,888
1,CESDE BOGOTÁ,1341,TÉCNICO LABORAL COMO AUXILIAR ADMINISTRATIVO,54890,56.75,90,"6,232,522.68",700,Grupos cerrados,"454,974,155.57",73.00,90
2,MEDISED INSTITUCION DE EDUCACION PARA EL TRABA...,1341,TÉCNICO LABORAL EN AUXILIAR ADMINISTRATIVO,17507,32.38,100,"6,839,631.92",1160,Grupos cerrados,0.00,0.00,100
3,FEE ESTUDIO EMPRESARIAL - CHAPINERO,1345,TÉCNICO LABORAL EN AUXILIAR ADMINISTRATIVO EN ...,43512,56.25,120,"6,504,271.65",1600,Grupos cerrados,0.00,0.00,420
4,FEE ESTUDIO EMPRESARIAL - TEUSAQUILLO,1345,TÉCNICO LABORAL EN AUXILIAR ADMINISTRATIVO EN ...,41020,56.25,120,"6,652,941.40",1600,Grupos cerrados,0.00,0.00,612
5,FEE ESTUDIO EMPRESARIAL - CHAPINERO,3315,TÉCNICO LABORAL EN AUXILIAR EN SERVICIOS FARMA...,43511,56.25,120,"6,194,543.61",1600,Grupos cerrados,0.00,0.00,420
6,FEE ESTUDIO EMPRESARIAL - TEUSAQUILLO,3315,TÉCNICO LABORAL EN AUXILIAR EN SERVICIOS FARMA...,41022,56.25,120,"6,652,941.40",1600,Grupos cerrados,0.00,0.00,612
7,KUEPA EDUTECH,6322,TÉCNICO LABORAL EN AUXILIAR EN MERCADEO Y VENTAS,42559,88.75,40,"4,810,454.10",720,Grupos cerrados,"4,810,454.10",1.00,277
8,KUEPA EDUTECH,6334,TÉCNICO LABORAL EN SERVICIOS TURISTICOS Y HOTE...,42560,88.75,20,"4,810,454.10",720,Grupos cerrados,0.00,0.00,69
9,CESDE BOGOTÁ,8325,TÉCNICO LABORAL COMO AUXILIAR EN SISTEMAS,54885,56.75,60,"5,406,154.01",694,Grupos cerrados,0.00,0.00,60
